# Span tables

In this example, you will learn how to manipulate various objects and get an overview of some core functionality of AutoRobot. First make sure the module can be imported from this notebook. If that's the case, the following cell should terminate whitout errors.

In [1]:
import autorobot as ar

Now the module needs initialization. This mainly consist of getting hold of an RSA application instance, which is then used to open a new project.

In [2]:
import math

rb = ar.initialize(
    unit_force=1e3,  # Forces in kN in AutoRobot, N in RSA API
    unit_angle_in=math.pi/180.,  # Input angles in degrees in AutoRobot, rad in RSA API
    unit_section=1e-3  # Section dimensions in mm, m in RSA API
)
rb.new('FRAME_2D')

For our span table, the main paramters are going to be the section size and the material. These values are defined at the top of the script for easier editing later.

In [3]:
section_name = 'UC 305x305x97'
material_name = 'S355'

# Looking-up section and material databases
current_section = rb.sections.get(section_name)
current_material = rb.materials.get(material_name)

# Make sure we have a section and material defined
assert(current_section and current_material)

# Repace the default material for our section
current_section.material = current_material


Now we define a few more parameters: minimum and maximum length, length increment and vertical spacing of beams.

In [4]:
# Using RSA units, length in meter
min_length = 2.
max_length = 6.
increment = 1.

vert_spacing = 1.

# Checking interval constants make sense
assert(min_length < max_length and increment > 0)

def to_mm(length):
    """Converts length to mm and truncates value to lower integer.
    """
    return int(length * 1e3)

def get_beam_lengths(min_len, max_len, increment):
    
    def get_range_params(min_len, max_len, increment):
        """Get parameters to obtain beam length through builtin *range*.
        """
        return map(to_mm, (min_len, max_len, increment))
    
    _range = range(*get_range_params(min_len, max_len, increment))
    return [length / 1e3 for length in _range] + [max_len]

beam_lengths = get_beam_lengths(min_length, max_length, increment)

print(f'Beam lengths: {beam_lengths}')

Beam lengths: [2.0, 3.0, 4.0, 5.0, 6.0]


The nodes and bars can now be created. `with rb.nodes, rb.bars:` is not mandatory, it just groups the operations on the node or bar server for effienciency.

In [5]:
geometry = []

with rb.nodes, rb.bars:
    for i, length in enumerate(beam_lengths):
        start = rb.nodes.create(0., vert_spacing * i, 0.)
        end = rb.nodes.create(length, vert_spacing * i, 0.)
        bar = rb.bars.create(start, end)
        geometry.append((bar, start, end))

We now need to group the bars and nodes to be able to select them. 

In [6]:
sel = []
for lst in zip(*geometry):
    number_strings = (str(el.Number) for el in lst)
    sel.append(' '.join(number_strings))

bar_sel, start_sel, end_sel = sel

print(f"All bars selection: \t'{bar_sel}'")
print(f"Start node selection: \t'{start_sel}'")
print(f"End node selection: \t'{end_sel}'")

All bars selection: 	'1 2 3 4 5'
Start node selection: 	'1 3 5 7 9'
End node selection: 	'2 4 6 8 10'


Let's now apply some supports to the nodes and a section property to the bars.

In [7]:
support_xxxfff = rb.supports.create('xxxfff', '111000')  # A pin support
support_fxxfff = rb.supports.create('fxxfff', '011000')  # A roller in the global X axis

rb.supports.set(support_xxxfff, start_sel)
rb.supports.set(support_fxxfff, end_sel)

rb.bars.set_section(current_section, bar_sel)

Now let's introduce some loads cases.

In [8]:
case_1_sw = rb.cases.create_case(1, 'Self-weight', 'PERM', 'LINEAR')

case_2_udl = rb.cases.create_case(2, 'UDL', 'IMPOSED', 'LINEAR')

And some associated loads.

In [9]:
rec_1 = case_1_sw.add_self_weight(desc='self-weight')

rec_2 = []
for bar in rb.bars.select(bar_sel):
    rec_2.append(case_2_udl.add_bar_udl(bar.Number, fz=-10., desc=f'udl bar {bar.Number}'))

Now we're ready to launch the calculation of the structure.

In [10]:
rb.calc_engine.calculate()

In [99]:
from RobotOM import (
    IRDimCalcEngine,
    IRDimAllRes,
    IRDimDetailedRes,
    IRDimStream,
    IRDimCalcParam,
    IRDimCalcConf,
    IRDimCalcParamVerifType,
    IRDimCalcParamLimitStateType
)

engine = IRDimCalcEngine(rb.steel_design.CalculEngine)
param = IRDimCalcParam(engine.GetCalcParam())
conf = IRDimCalcConf(engine.GetCalcConf())

stream = IRDimStream(rb.steel_design.Connection.GetStream())
stream.Clear()
stream.WriteText(bar_sel)
param.SetObjsList(IRDimCalcParamVerifType.I_DCPVT_MEMBERS_VERIF, stream)
stream.Clear()
param.SetLimitState(IRDimCalcParamLimitStateType.I_DCPLST_ULTIMATE, 1)
param.SetLimitState(IRDimCalcParamLimitStateType.I_DCPLST_SERVICEABILITY, 1)

stream.WriteText('1 2')
param.SetLoadsList(stream)

engine.SetCalcConf(conf)
engine.SetCalcParam(param)

engine.Solve(None)

In [100]:
for i in range(5):
    res = IRDimAllRes(engine.Results())
    det_res = IRDimDetailedRes(res.Get(rb.bars.get(i + 1).Number))

    print(det_res.GovernCaseName)
    print(det_res.Ratio)

2 UDL
0.013871166491391178
2 UDL
0.02241295697134039
2 UDL
0.042520107052147274
2 UDL
0.07148264430476993
2 UDL
0.11141987459431034


In [103]:
from RobotOM import IRobotPredefinedLabel, IRobotLabel, IRDimMembDefData, IRDimMembDefDeflDataType

name = rb.structure.Labels.GetPredefinedName(IRobotPredefinedLabel.I_PL_MEMBER_TYPE_BEAM)

label = IRobotLabel(rb.structure.Labels.Get(ar.constants.RLabelType.I_LT_MEMBER_TYPE, name))

data = IRDimMembDefData(label.Data)

for attr in dir(data):
    if attr[0].isupper():
        print(attr, getattr(data, attr))
        
print(data.DeflYZRelLimit(IRDimMembDefDeflDataType.I_DMDDDT_DEFL_Z))

print(dir(IRDimMembDefDeflDataType))

CantileverMode 0
CodeParams System.__ComObject
DeflYZRelLimit <bound method 'DeflYZRelLimit'>
DisplXYRelLimit <bound method 'DisplXYRelLimit'>
Equals <bound method 'Equals'>
GetHashCode <bound method 'GetHashCode'>
GetType <bound method 'GetType'>
GetUserInitDefl <bound method 'GetUserInitDefl'>
InitDeflType I_DMDIDT_LACK
IntPsClear <bound method 'IntPsClear'>
IntPsCoordNum <bound method 'IntPsCoordNum'>
IntPsCoordValueType <bound method 'IntPsCoordValueType'>
IntPtAddOption <bound method 'IntPtAddOption'>
IntPtValues <bound method 'IntPtValues'>
IsDeflectionYZ <bound method 'IsDeflectionYZ'>
IsDisplacementXY <bound method 'IsDisplacementXY'>
IsStructureSwayYZ <bound method 'IsStructureSwayYZ'>
LengthYZUV <bound method 'LengthYZUV'>
Name Beam
PipeMode 0
RangeCoords <bound method 'RangeCoords'>
RangeMoments 0
Retrieve <bound method 'Retrieve'>
RetrieveAdjoinParamsYZ <bound method 'RetrieveAdjoinParamsYZ'>
RetrieveExpIntPs <bound method 'RetrieveExpIntPs'>
SetDeflYZRelLimit <bound method

In [35]:
from RobotOM import IRobotCodeType

rb.Project.Preferences.GetActiveCode(IRobotCodeType(2))

'EN 1995-1:2004/A2:2014'

In [102]:
from RobotOM import IRobotBarSectionDataValue, IRDimCodeResE32_1, IRDimCodeResE32_2, IRDimCodeResE32_3, IRDimCodeResE32_4

# for i in range(38):
#     print(i, ': ', current_section.data.GetValue(IRobotBarSectionDataValue(i, True)))

code_res_1 = IRDimCodeResE32_1(det_res.CodeResults)
code_res_2 = IRDimCodeResE32_2(det_res.CodeResults)
code_res_3 = IRDimCodeResE32_3(det_res.CodeResults)
code_res_4 = IRDimCodeResE32_4(det_res.CodeResults)
# print(dir(code_res_1))
# print(dir(code_res_2))
# print(dir(code_res_3))
# print(dir(code_res_4))

for attr in dir(det_res):
    if attr[0].isupper():
        print(attr, getattr(det_res, attr))


for attr in dir(code_res_1):
    if attr[0].isupper():
        print(attr, getattr(code_res_1, attr))

for attr in dir(code_res_2):
    if attr[0].isupper():
        print(attr, getattr(code_res_2, attr))



CodeResults System.__ComObject
EffDef RobotOM.RDimEffDefClass
Equals <bound method 'Equals'>
GetGovernCase_uy <bound method 'GetGovernCase_uy'>
GetGovernCase_uz <bound method 'GetGovernCase_uz'>
GetGovernCase_vx <bound method 'GetGovernCase_vx'>
GetGovernCase_vy <bound method 'GetGovernCase_vy'>
GetHashCode <bound method 'GetHashCode'>
GetType <bound method 'GetType'>
GovernCaseName 2 UDL
GovernCaseName_uy 1 Self-weight
GovernCaseName_uz 2 UDL
GovernCaseName_vx 
GovernCaseName_vy 
IsLimitStateService_uy 1
IsLimitStateService_uz 1
IsLimitStateService_vx 0
IsLimitStateService_vy 0
IsLimitStateUltimate 1
Lay 44.61067539895089
Laz 77.84030640030016
MatDef RobotOM.RDimMatDefClass
MaterialName S355
MembDef RobotOM.RDimMembDefClass
MemberUsrNo 5
ProfDef RobotOM.RDimProfDefClass
ProfileName UC 305x305x97
Ratio 0.11141987459431034
RelLimit_uy 0.03
RelLimit_uz 0.03
RelLimit_vx 0.0
RelLimit_vy -1.0
ResOfCalc I_DMCRV_CORRECT
ToString <bound method 'ToString'>
UY 0.0
UZ 0.003699643738010413
AxForcE